In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

port_waiting_df_improve_plus = pd.read_pickle('port_waiting_df_improve_plus.pkl') 
working_vessel_cnt_mean_improve = pd.read_pickle('working_vessel_cnt_mean_improve.pkl')

In [2]:
port_waiting_df_improve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53371 entries, 0 to 53370
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   PORT_CD       53371 non-null  object         
 1   VSL_ID        53371 non-null  object         
 2   ATA_DT        53371 non-null  datetime64[ns] 
 3   ATB_DT        53371 non-null  datetime64[ns] 
 4   ATD_DT        53371 non-null  datetime64[ns] 
 5   SHIP_TYPE     53371 non-null  object         
 6   working_time  53371 non-null  timedelta64[ns]
dtypes: datetime64[ns](3), object(3), timedelta64[ns](1)
memory usage: 2.9+ MB


In [11]:
port_waiting_df_improve = port_waiting_df_improve[['PORT_CD', 'VSL_ID', 'ATA_DT', 'ATB_DT', 'ATD_DT', 'SHIP_TYPE', 'working_time']]

In [121]:
# 작업과 대기 중인 선박은 따로 데이터 프레임으로 
def show_me_the_vessel_count(port_name, date):
    # 전달받은 port: 데이터프레임 생성
    target_port_df = port_waiting_df_improve[port_waiting_df_improve['PORT_CD']==port_name]
    
    # 전달받은 date: string to datetime type 변환
    target_time = pd.to_datetime(date)

    # 
    finding_waiting_df = target_port_df[(target_port_df['ATA_DT'] < target_time) & (target_port_df['ATB_DT'] > target_time)]

    ship_list=[] # 선박종류별 개수가 들어감
    type_list = [] # 선박종류별 고유값이 들어감
    
    for ship_type in finding_waiting_df['SHIP_TYPE'].unique():
        type_list.append(ship_type) # 현시점의 대기중인 선박의 종류를 구함

        finding_waiting_df_type = finding_waiting_df[finding_waiting_df['SHIP_TYPE'] == ship_type]
        ship_list.append(len(finding_waiting_df_type)) # 선박 종류별로 대기중인 선박의 개수를 구함

    # total 추가
    type_list.append('Total') 
    ship_list.append(np.sum(ship_list))
    wait_result_df = pd.DataFrame(data={'SHIP_TYPE': type_list , '대기선박 대수': ship_list})

    
################# 작업선박 찾기
    finding_working_df = target_port_df[(target_port_df['ATB_DT'] < target_time) & (target_port_df['ATD_DT'] > target_time)]

    ship_list=[] # 선박종류별 개수가 들어감
    type_list = [] # 선박종류별 고유값이 들어감
    
    for ship_type in finding_working_df['SHIP_TYPE'].unique():
        type_list.append(ship_type) # 현시점의 작업중인 선박의 종류를 구함

        finding_working_df_type = finding_working_df[finding_working_df['SHIP_TYPE'] == ship_type]
        ship_list.append(len(finding_working_df_type)) # 선박 종류별로 작업중인 선박의 개수를 구함

    # total 추가
    type_list.append('Total') 
    ship_list.append(np.sum(ship_list))
    work_result_df = pd.DataFrame(data={'SHIP_TYPE': type_list , '작업선박 대수': ship_list})

    s = float(working_vessel_cnt_mean_improve[working_vessel_cnt_mean_improve['PORT_CD'] == port_name]['vessel_cnt'])
    rest = ship_list[-1] - s

    if rest > 0:
        comment = f'평균보다 {rest:.0f}대 많은 선박이 작업하고 있습니다.'
    elif rest == 0:
        comment = f'평균{rest:.0f}과 같은 수의 선박이 작업하고 있습니다.'
    else:
        comment = f'평균보다 {abs(rest):.0f}대 적은 선박이 작업하고 있습니다.'
    
    print('실시간 항구 현황은 다음과 같습니다.')
    return wait_result_df, work_result_df, comment

In [122]:
def port_congestion_index(port, date):
    target_port = port_waiting_df_improve[port_waiting_df_improve['PORT_CD'] == port]
    
    # start_time: 전달받은 date로부터 24시간 전
    start_time = pd.to_datetime(date) - pd.Timedelta(days=1)
    
    # date로부터 지난 24시간 동안 입항한(=ATA_DT) 선박들 추출
    lambda_df = target_port[(target_port['ATA_DT'] <= pd.to_datetime(date)) & (target_port['ATA_DT'] >= start_time)]    
    # date로부터 지난 24시간 동안 작업을 마친(=ATD_DT) 선박 추출
    mu_df = target_port[(target_port['ATD_DT'] <= pd.to_datetime(date)) & (target_port['ATD_DT'] >= start_time)]

    # 작업중인 선박의 대수가 0대일 때 (항구 shutdown) 반환값
    if len(mu_df) == 0:
        print(f'지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.')
        print()
        return print('혼잡도 반환 불가!🙁', '\n사유: 작업중이었던 선박이 없습니다.')
        
    else:
        # Lambda: 선박 arrival rate
        Lambda = round(len(lambda_df)/24, 2)

        # Mu: 작업 rate (Mu_invers: 평균 작업시간)
        Mu_pre = mu_df['working_time'].mean() 
        hour = int(str(Mu_pre).split(' ')[2].split(':')[0]) # 예) '0 days 10:11:27.833333333' 에서 10을 추출
        minutes = round(int(str(Mu_pre).split(' ')[2].split(':')[1])/60, 2) # 11분에서 60으로 나눔
        Mu_inverse = hour + minutes # 시간 단위로 추출

        # s: 항구별 평균 선박 수용량
        s = float(working_vessel_cnt_mean_improve[working_vessel_cnt_mean_improve['PORT_CD'] == port]['vessel_cnt'])

        # roh: 항구 utilization rate (항구작업효율)
        roh = np.round((Lambda*Mu_inverse)/s, 2)

        Mu = 1/Mu_inverse

        # roh값에 따른 결과값 반환
        print(f'지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.')
        print()
        if roh < 0.25:
            return print('혼잡도: 매우여유','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))
        elif 0.25 <= roh < 0.75:
            return print('혼잡도: 여유','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))
        elif 0.75 <= roh < 1.25:
            return print('혼잡도: 보통','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))
        elif 1.25 <= roh < 1.75:
            return print('혼잡도: 혼잡','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))
        else:
            return print('혼잡도: 매우혼잡','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))

In [123]:
port = 'KRPUS'
datetime = '2023-08-09 22:00:05'
port_congestion_index(port, datetime)
print()
df1, df2, comment = show_me_the_vessel_count(port, datetime)
display(df1, df2)
print(comment)

지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.

혼잡도 반환 불가!🙁 
사유: 작업중이었던 선박이 없습니다.

실시간 항구 현황은 다음과 같습니다.


,SHIP_TYPE,대기선박 대수
0,Total,0.0


,SHIP_TYPE,작업선박 대수
0,Total,0.0


평균보다 17대 적은 선박이 작업하고 있습니다.


In [124]:
port = 'HKHKG'
datetime = '2023-04-17 22:30:05'
port_congestion_index(port, datetime)
print()
df1, df2, comment = show_me_the_vessel_count(port, datetime)
display(df1, df2)
print(comment)

지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.

혼잡도: 보통 
작업효율: 1.1 | 평균 작업시간: 0 days 11:46:18

실시간 항구 현황은 다음과 같습니다.


,SHIP_TYPE,대기선박 대수
0,CONTAINER,6
1,GENERAL_CARGO,16
2,UNKNOWN,4
3,Total,26


,SHIP_TYPE,작업선박 대수
0,CONTAINER,9
1,GENERAL_CARGO,12
2,UNKNOWN,4
3,Total,25


평균보다 13대 적은 선박이 작업하고 있습니다.


In [125]:
port = 'JPOSA'
datetime = '2024-04-17 22:30:05'
port_congestion_index(port, datetime)
print()
df1, df2, comment = show_me_the_vessel_count(port, datetime)
display(df1, df2)
print(comment)

지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.

혼잡도 반환 불가!🙁 
사유: 작업중이었던 선박이 없습니다.

실시간 항구 현황은 다음과 같습니다.


,SHIP_TYPE,대기선박 대수
0,Total,0.0


,SHIP_TYPE,작업선박 대수
0,Total,0.0


평균보다 4대 적은 선박이 작업하고 있습니다.


In [12]:
port_waiting_df_improve.to_pickle('port_waiting_df_improve.pkl')

# 데이터 복사 및 7개월 더하기(ata, atb, atd)

In [110]:
port_waiting_df_improve

,PORT_CD,VSL_ID,ATA_DT,ATB_DT,ATD_DT,SHIP_TYPE,working_time
0,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-03-07 06:50:02,2023-03-07 09:54:33,2023-03-07 20:35:24,CONTAINER,0 days 10:40:51
1,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-04-01 06:36:33,2023-04-01 09:37:49,2023-04-02 03:45:09,CONTAINER,0 days 18:07:20
2,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-05-27 09:15:54,2023-05-27 11:57:40,2023-05-28 03:05:05,CONTAINER,0 days 15:07:25
3,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-07-06 19:35:03,2023-07-06 22:35:01,2023-07-07 19:55:03,CONTAINER,0 days 21:20:02
4,JPTYO,006f1ec5-89e2-3026-b377-6a5d5563eb9a,2023-03-25 03:15:10,2023-03-25 05:55:03,2023-03-25 20:55:06,CONTAINER,0 days 15:00:03
...,...,...,...,...,...,...,...
53366,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2023-09-02 06:45:03,2023-09-02 07:12:35,2023-09-03 10:35:22,GENERAL_TANKER,1 days 03:22:47
53367,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2023-09-18 11:15:45,2023-09-18 12:38:51,2023-09-18 18:35:04,GENERAL_TANKER,0 days 05:56:13
53368,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2023-09-22 22:15:12,2023-09-22 22:25:19,2023-09-23 08:15:14,GENERAL_TANKER,0 days 09:49:55
53369,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2023-09-24 14:15:22,2023-09-24 14:48:47,2023-09-24 21:05:03,GENERAL_TANKER,0 days 06:16:16


In [95]:
port_waiting_df_improve_2 = port_waiting_df_improve.copy()

In [97]:
port_waiting_df_improve_2['ATA_DT'] = port_waiting_df_improve['ATA_DT'] + pd.Timedelta(weeks=30)
port_waiting_df_improve_2['ATD_DT'] = port_waiting_df_improve['ATD_DT'] + pd.Timedelta(weeks=30)
port_waiting_df_improve_2['ATB_DT'] = port_waiting_df_improve['ATB_DT'] + pd.Timedelta(weeks=30)

In [101]:
print(" ATA_DT 최소:",port_waiting_df_improve_2['ATA_DT'].max())
print(" ATD_DT 최대:",port_waiting_df_improve_2['ATD_DT'].max())

 ATA_DT 최소: 2024-04-27 22:35:32
 ATD_DT 최대: 2024-05-24 02:56:15


In [127]:
port_waiting_df_improve_plus = pd.concat([port_waiting_df_improve, port_waiting_df_improve_2], ignore_index=True)

In [128]:
port_waiting_df_improve_plus.to_pickle('port_waiting_df_improve_plus.pkl')

In [129]:
# 작업과 대기 중인 선박은 따로 데이터 프레임으로 
def show_me_the_vessel_count(port_name, date):
    # 전달받은 port: 데이터프레임 생성
    target_port_df = port_waiting_df_improve_plus[port_waiting_df_improve_plus['PORT_CD']==port_name]
    
    # 전달받은 date: string to datetime type 변환
    target_time = pd.to_datetime(date)

    # 
    finding_waiting_df = target_port_df[(target_port_df['ATA_DT'] < target_time) & (target_port_df['ATB_DT'] > target_time)]

    ship_list=[] # 선박종류별 개수가 들어감
    type_list = [] # 선박종류별 고유값이 들어감
    
    for ship_type in finding_waiting_df['SHIP_TYPE'].unique():
        type_list.append(ship_type) # 현시점의 대기중인 선박의 종류를 구함

        finding_waiting_df_type = finding_waiting_df[finding_waiting_df['SHIP_TYPE'] == ship_type]
        ship_list.append(len(finding_waiting_df_type)) # 선박 종류별로 대기중인 선박의 개수를 구함

    # total 추가
    type_list.append('Total') 
    ship_list.append(np.sum(ship_list))
    wait_result_df = pd.DataFrame(data={'SHIP_TYPE': type_list , '대기선박 대수': ship_list})

    
################# 작업선박 찾기
    finding_working_df = target_port_df[(target_port_df['ATB_DT'] < target_time) & (target_port_df['ATD_DT'] > target_time)]

    ship_list=[] # 선박종류별 개수가 들어감
    type_list = [] # 선박종류별 고유값이 들어감
    
    for ship_type in finding_working_df['SHIP_TYPE'].unique():
        type_list.append(ship_type) # 현시점의 작업중인 선박의 종류를 구함

        finding_working_df_type = finding_working_df[finding_working_df['SHIP_TYPE'] == ship_type]
        ship_list.append(len(finding_working_df_type)) # 선박 종류별로 작업중인 선박의 개수를 구함

    # total 추가
    type_list.append('Total') 
    ship_list.append(np.sum(ship_list))
    work_result_df = pd.DataFrame(data={'SHIP_TYPE': type_list , '작업선박 대수': ship_list})

    s = float(working_vessel_cnt_mean_improve[working_vessel_cnt_mean_improve['PORT_CD'] == port_name]['vessel_cnt'])
    rest = ship_list[-1] - s

    if rest > 0:
        comment = f'평균보다 {rest:.0f}대 많은 선박이 작업하고 있습니다.'
    elif rest == 0:
        comment = f'평균{rest:.0f}과 같은 수의 선박이 작업하고 있습니다.'
    else:
        comment = f'평균보다 {abs(rest):.0f}대 적은 선박이 작업하고 있습니다.'
    
    print('실시간 항구 현황은 다음과 같습니다.')
    return wait_result_df, work_result_df, comment

In [130]:
def port_congestion_index(port, date):
    target_port = port_waiting_df_improve_plus[port_waiting_df_improve_plus['PORT_CD'] == port]
    
    # start_time: 전달받은 date로부터 24시간 전
    start_time = pd.to_datetime(date) - pd.Timedelta(days=1)
    
    # date로부터 지난 24시간 동안 입항한(=ATA_DT) 선박들 추출
    lambda_df = target_port[(target_port['ATA_DT'] <= pd.to_datetime(date)) & (target_port['ATA_DT'] >= start_time)]    
    # date로부터 지난 24시간 동안 작업을 마친(=ATD_DT) 선박 추출
    mu_df = target_port[(target_port['ATD_DT'] <= pd.to_datetime(date)) & (target_port['ATD_DT'] >= start_time)]

    # 작업중인 선박의 대수가 0대일 때 (항구 shutdown) 반환값
    if len(mu_df) == 0:
        print(f'지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.')
        print()
        return print('혼잡도 반환 불가!🙁', '\n사유: 작업중이었던 선박이 없습니다.')
        
    else:
        # Lambda: 선박 arrival rate
        Lambda = round(len(lambda_df)/24, 2)

        # Mu: 작업 rate (Mu_invers: 평균 작업시간)
        Mu_pre = mu_df['working_time'].mean() 
        hour = int(str(Mu_pre).split(' ')[2].split(':')[0]) # 예) '0 days 10:11:27.833333333' 에서 10을 추출
        minutes = round(int(str(Mu_pre).split(' ')[2].split(':')[1])/60, 2) # 11분에서 60으로 나눔
        Mu_inverse = hour + minutes # 시간 단위로 추출

        # s: 항구별 평균 선박 수용량
        s = float(working_vessel_cnt_mean_improve[working_vessel_cnt_mean_improve['PORT_CD'] == port]['vessel_cnt'])

        # roh: 항구 utilization rate (항구작업효율)
        roh = np.round((Lambda*Mu_inverse)/s, 2)

        Mu = 1/Mu_inverse

        # roh값에 따른 결과값 반환
        print(f'지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.')
        print()
        if roh < 0.25:
            return print('혼잡도: 매우여유','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))
        elif 0.25 <= roh < 0.75:
            return print('혼잡도: 여유','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))
        elif 0.75 <= roh < 1.25:
            return print('혼잡도: 보통','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))
        elif 1.25 <= roh < 1.75:
            return print('혼잡도: 혼잡','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))
        else:
            return print('혼잡도: 매우혼잡','\n' '작업효율:',roh, '| ' '평균 작업시간:', Mu_pre.round(freq='s'))

In [134]:
port = 'KRPUS'
datetime = '2023-12-09 22:00:05'
port_congestion_index(port, datetime)
print()
df1, df2, comment = show_me_the_vessel_count(port, datetime)
display(df1, df2)
print(comment)

지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.

혼잡도: 보통 
작업효율: 1.04 | 평균 작업시간: 0 days 15:32:49

실시간 항구 현황은 다음과 같습니다.


,SHIP_TYPE,대기선박 대수
0,TANKER_PRODUCT,1
1,Total,1


,SHIP_TYPE,작업선박 대수
0,CONTAINER,13
1,UNKNOWN,2
2,GENERAL_CARGO,4
3,ROLL_ON_ROLL_OFF,1
4,VEHICLE_PASSENGER,1
5,Total,21


평균보다 4대 많은 선박이 작업하고 있습니다.


In [136]:
port = 'HKHKG'
datetime = '2024-04-17 22:30:05'
port_congestion_index(port, datetime)
print()
df1, df2, comment = show_me_the_vessel_count(port, datetime)
display(df1, df2)
print(comment)

지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.

혼잡도: 보통 
작업효율: 1.09 | 평균 작업시간: 0 days 08:23:18

실시간 항구 현황은 다음과 같습니다.


,SHIP_TYPE,대기선박 대수
0,CONTAINER,6
1,GENERAL_CARGO,21
2,UNKNOWN,2
3,Total,29


,SHIP_TYPE,작업선박 대수
0,CONTAINER,14
1,GENERAL_CARGO,26
2,UNKNOWN,2
3,Total,42


평균보다 4대 많은 선박이 작업하고 있습니다.


In [137]:
port = 'JPOSA'
datetime = '2024-01-01 22:30:05'
port_congestion_index(port, datetime)
print()
df1, df2, comment = show_me_the_vessel_count(port, datetime)
display(df1, df2)
print(comment)

지난 24시간 기준 항구의 혼잡도는 다음과 같습니다.

혼잡도: 혼잡 
작업효율: 1.53 | 평균 작업시간: 0 days 09:52:17

실시간 항구 현황은 다음과 같습니다.


,SHIP_TYPE,대기선박 대수
0,CONTAINER,5
1,Total,5


,SHIP_TYPE,작업선박 대수
0,CONTAINER,3
1,Total,3


평균보다 1대 적은 선박이 작업하고 있습니다.


In [7]:
port_waiting_df_improve_plus_test = port_waiting_df_improve_plus.copy()
port_waiting_df_improve_plus_test = port_waiting_df_improve_plus_test.drop('working_time', axis=1)

In [9]:
port_waiting_df_improve_plus_test

,PORT_CD,VSL_ID,ATA_DT,ATB_DT,ATD_DT,SHIP_TYPE
0,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-03-07 06:50:02,2023-03-07 09:54:33,2023-03-07 20:35:24,CONTAINER
1,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-04-01 06:36:33,2023-04-01 09:37:49,2023-04-02 03:45:09,CONTAINER
2,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-05-27 09:15:54,2023-05-27 11:57:40,2023-05-28 03:05:05,CONTAINER
3,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-07-06 19:35:03,2023-07-06 22:35:01,2023-07-07 19:55:03,CONTAINER
4,JPTYO,006f1ec5-89e2-3026-b377-6a5d5563eb9a,2023-03-25 03:15:10,2023-03-25 05:55:03,2023-03-25 20:55:06,CONTAINER
...,...,...,...,...,...,...
106737,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-03-30 06:45:03,2024-03-30 07:12:35,2024-03-31 10:35:22,GENERAL_TANKER
106738,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-15 11:15:45,2024-04-15 12:38:51,2024-04-15 18:35:04,GENERAL_TANKER
106739,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-19 22:15:12,2024-04-19 22:25:19,2024-04-20 08:15:14,GENERAL_TANKER
106740,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-21 14:15:22,2024-04-21 14:48:47,2024-04-21 21:05:03,GENERAL_TANKER


# 데이터 csv

In [17]:
df = pd.read_pickle('port_waiting_df_improve_plus.pkl')
df ## 전처리 개선 + 기간연장

,PORT_CD,VSL_ID,ATA_DT,ATB_DT,ATD_DT,SHIP_TYPE,working_time
0,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-03-07 06:50:02,2023-03-07 09:54:33,2023-03-07 20:35:24,CONTAINER,0 days 10:40:51
1,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-04-01 06:36:33,2023-04-01 09:37:49,2023-04-02 03:45:09,CONTAINER,0 days 18:07:20
2,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-05-27 09:15:54,2023-05-27 11:57:40,2023-05-28 03:05:05,CONTAINER,0 days 15:07:25
3,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-07-06 19:35:03,2023-07-06 22:35:01,2023-07-07 19:55:03,CONTAINER,0 days 21:20:02
4,JPTYO,006f1ec5-89e2-3026-b377-6a5d5563eb9a,2023-03-25 03:15:10,2023-03-25 05:55:03,2023-03-25 20:55:06,CONTAINER,0 days 15:00:03
...,...,...,...,...,...,...,...
106737,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-03-30 06:45:03,2024-03-30 07:12:35,2024-03-31 10:35:22,GENERAL_TANKER,1 days 03:22:47
106738,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-15 11:15:45,2024-04-15 12:38:51,2024-04-15 18:35:04,GENERAL_TANKER,0 days 05:56:13
106739,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-19 22:15:12,2024-04-19 22:25:19,2024-04-20 08:15:14,GENERAL_TANKER,0 days 09:49:55
106740,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-21 14:15:22,2024-04-21 14:48:47,2024-04-21 21:05:03,GENERAL_TANKER,0 days 06:16:16


In [18]:
df = df.drop('working_time', axis=1)
df ## 오라클에서 쓸 수 없는 타임델타 타입인 working_time 컬럼 드랍

,PORT_CD,VSL_ID,ATA_DT,ATB_DT,ATD_DT,SHIP_TYPE
0,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-03-07 06:50:02,2023-03-07 09:54:33,2023-03-07 20:35:24,CONTAINER
1,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-04-01 06:36:33,2023-04-01 09:37:49,2023-04-02 03:45:09,CONTAINER
2,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-05-27 09:15:54,2023-05-27 11:57:40,2023-05-28 03:05:05,CONTAINER
3,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-07-06 19:35:03,2023-07-06 22:35:01,2023-07-07 19:55:03,CONTAINER
4,JPTYO,006f1ec5-89e2-3026-b377-6a5d5563eb9a,2023-03-25 03:15:10,2023-03-25 05:55:03,2023-03-25 20:55:06,CONTAINER
...,...,...,...,...,...,...
106737,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-03-30 06:45:03,2024-03-30 07:12:35,2024-03-31 10:35:22,GENERAL_TANKER
106738,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-15 11:15:45,2024-04-15 12:38:51,2024-04-15 18:35:04,GENERAL_TANKER
106739,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-19 22:15:12,2024-04-19 22:25:19,2024-04-20 08:15:14,GENERAL_TANKER
106740,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-21 14:15:22,2024-04-21 14:48:47,2024-04-21 21:05:03,GENERAL_TANKER


In [20]:
df.to_csv('port_waiting_df_improve_plus.csv', index=True) ## csv로 변환 + index 컬럼 생성

In [21]:
df1 = pd.read_csv('port_waiting_df_improve_plus.csv') 
df1

,Unnamed: 0,PORT_CD,VSL_ID,ATA_DT,ATB_DT,ATD_DT,SHIP_TYPE
0,0,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-03-07 06:50:02,2023-03-07 09:54:33,2023-03-07 20:35:24,CONTAINER
1,1,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-04-01 06:36:33,2023-04-01 09:37:49,2023-04-02 03:45:09,CONTAINER
2,2,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-05-27 09:15:54,2023-05-27 11:57:40,2023-05-28 03:05:05,CONTAINER
3,3,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-07-06 19:35:03,2023-07-06 22:35:01,2023-07-07 19:55:03,CONTAINER
4,4,JPTYO,006f1ec5-89e2-3026-b377-6a5d5563eb9a,2023-03-25 03:15:10,2023-03-25 05:55:03,2023-03-25 20:55:06,CONTAINER
...,...,...,...,...,...,...,...
106737,106737,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-03-30 06:45:03,2024-03-30 07:12:35,2024-03-31 10:35:22,GENERAL_TANKER
106738,106738,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-15 11:15:45,2024-04-15 12:38:51,2024-04-15 18:35:04,GENERAL_TANKER
106739,106739,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-19 22:15:12,2024-04-19 22:25:19,2024-04-20 08:15:14,GENERAL_TANKER
106740,106740,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-21 14:15:22,2024-04-21 14:48:47,2024-04-21 21:05:03,GENERAL_TANKER


In [22]:
df1 = df1.rename(columns={'Unnamed: 0': 'seq'}) # 컬럼명은 seq로 변환
df1

,seq,PORT_CD,VSL_ID,ATA_DT,ATB_DT,ATD_DT,SHIP_TYPE
0,0,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-03-07 06:50:02,2023-03-07 09:54:33,2023-03-07 20:35:24,CONTAINER
1,1,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-04-01 06:36:33,2023-04-01 09:37:49,2023-04-02 03:45:09,CONTAINER
2,2,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-05-27 09:15:54,2023-05-27 11:57:40,2023-05-28 03:05:05,CONTAINER
3,3,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-07-06 19:35:03,2023-07-06 22:35:01,2023-07-07 19:55:03,CONTAINER
4,4,JPTYO,006f1ec5-89e2-3026-b377-6a5d5563eb9a,2023-03-25 03:15:10,2023-03-25 05:55:03,2023-03-25 20:55:06,CONTAINER
...,...,...,...,...,...,...,...
106737,106737,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-03-30 06:45:03,2024-03-30 07:12:35,2024-03-31 10:35:22,GENERAL_TANKER
106738,106738,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-15 11:15:45,2024-04-15 12:38:51,2024-04-15 18:35:04,GENERAL_TANKER
106739,106739,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-19 22:15:12,2024-04-19 22:25:19,2024-04-20 08:15:14,GENERAL_TANKER
106740,106740,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-21 14:15:22,2024-04-21 14:48:47,2024-04-21 21:05:03,GENERAL_TANKER


In [23]:
df1['seq'] = df1['seq'] + 1
df1 # seq 전체에 1씩 더함

,seq,PORT_CD,VSL_ID,ATA_DT,ATB_DT,ATD_DT,SHIP_TYPE
0,1,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-03-07 06:50:02,2023-03-07 09:54:33,2023-03-07 20:35:24,CONTAINER
1,2,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-04-01 06:36:33,2023-04-01 09:37:49,2023-04-02 03:45:09,CONTAINER
2,3,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-05-27 09:15:54,2023-05-27 11:57:40,2023-05-28 03:05:05,CONTAINER
3,4,JPTYO,001942e3-a2f0-3632-9ac8-f09b66aba6ec,2023-07-06 19:35:03,2023-07-06 22:35:01,2023-07-07 19:55:03,CONTAINER
4,5,JPTYO,006f1ec5-89e2-3026-b377-6a5d5563eb9a,2023-03-25 03:15:10,2023-03-25 05:55:03,2023-03-25 20:55:06,CONTAINER
...,...,...,...,...,...,...,...
106737,106738,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-03-30 06:45:03,2024-03-30 07:12:35,2024-03-31 10:35:22,GENERAL_TANKER
106738,106739,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-15 11:15:45,2024-04-15 12:38:51,2024-04-15 18:35:04,GENERAL_TANKER
106739,106740,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-19 22:15:12,2024-04-19 22:25:19,2024-04-20 08:15:14,GENERAL_TANKER
106740,106741,KRPUS,ddbc8df2-31b0-3bf9-93a8-0e59115bd17c,2024-04-21 14:15:22,2024-04-21 14:48:47,2024-04-21 21:05:03,GENERAL_TANKER


In [25]:
df1.to_csv('port_waiting_df_improve_plus.csv', index=False) # 다시 csv로 변환